In [10]:
import pandas as pd
import requests
import hmac
import hashlib
import base64
import time
import os
from typing import Dict, List
from dataclasses import dataclass
import logging
from tkinter import Tk, filedialog, simpledialog
import openpyxl

@dataclass
class NaverAPIConfig:
    API_KEY: str
    SECRET_KEY: str
    CUSTOMER_ID: str

@dataclass
class SearchVolume:
    total: int

class NaverSearchVolume:
    def __init__(self, config: NaverAPIConfig):
        self.config = config
        self.session = requests.Session()
        
        # Set up logging
        logging.basicConfig(filename='search_volume.log', level=logging.DEBUG)

    def get_naver_search_volume(self, keywords: List[str]) -> Dict[str, SearchVolume]:
        result = {k: SearchVolume(0) for k in keywords}  # 기본값 0으로 설정
        max_retries = 3  # 최대 재시도 횟수
        total_keywords = len(keywords)
        print(f"총 요청 키워드 수: {total_keywords}")
        
        try:
            for i in range(0, len(keywords), 5):  # 5개씩 요청 (API 제한 고려)
                sub_keywords = keywords[i:i + 5]
                print(f"처리 중: {i+1}/{total_keywords} 키워드 ({', '.join(sub_keywords)})")
                logging.debug(f"요청 중 키워드 그룹: {sub_keywords}")
                
                timestamp = str(int(time.time() * 1000))
                method = "GET"
                uri = "/keywordstool"
                signature = self.generate_signature(timestamp, method, uri)

                url = "https://api.naver.com" + uri
                headers = {
                    "X-API-KEY": self.config.API_KEY,
                    "X-CUSTOMER": self.config.CUSTOMER_ID,
                    "X-Timestamp": timestamp,
                    "X-Signature": signature,
                    "Content-Type": "application/json",
                    "Accept": "*/*"
                }
                params = {
                    "hintKeywords": ",".join(sub_keywords),
                    "showDetail": "1"
                }
                
                for attempt in range(max_retries):
                    try:
                        response = self.session.get(url, headers=headers, params=params)
                        response.raise_for_status()
                        data = response.json()
                        logging.debug(f"API 응답 데이터: {data}")
                        
                        for item in data.get("keywordList", []):
                            keyword, total_search_volume = self.process_search_data(item, sub_keywords)
                            if keyword:
                                result[keyword] = SearchVolume(total_search_volume)
                        break  # 성공하면 반복문 탈출
                    except requests.exceptions.RequestException as e:
                        logging.error(f"API 요청 중 오류 발생 (재시도 {attempt+1}/{max_retries}): {str(e)}")
                        if response.status_code == 429 and attempt < max_retries - 1:
                            print("429 오류 발생, 5초 후 재시도...")
                            time.sleep(5)  # 429 오류 발생 시 5초 대기 후 재시도
                        else:
                            raise e  # 다른 오류거나 최대 재시도 초과 시 예외 발생
                
                time.sleep(1)  # 요청 간 1초 대기
        except requests.RequestException as e:
            logging.error(f"API 요청 중 오류 발생: {str(e)}")
        except Exception as e:
            logging.error(f"일반 오류 발생: {str(e)}")
        return result

    def generate_signature(self, timestamp: str, method: str, uri: str) -> str:
        message = f"{timestamp}.{method}.{uri}"
        signature = hmac.new(self.config.SECRET_KEY.encode(), message.encode(), hashlib.sha256).digest()
        return base64.b64encode(signature).decode()
    
    def process_search_data(self, item: Dict, sub_keywords: List[str]) -> tuple:
        keyword = item.get("relKeyword")
        if keyword in sub_keywords:
            total_search_volume = self.parse_search_count(item.get("monthlyPcQcCnt", 0)) + self.parse_search_count(item.get("monthlyMobileQcCnt", 0))
            return keyword, total_search_volume
        return None, None
    
    @staticmethod
    def parse_search_count(count) -> int:
        if isinstance(count, str) and '<' in count:
            return 5  # 검색량이 '10 미만'인 경우 최소값 5로 설정
        return int(count)
    
    def process_excel(self):
        try:
            root = Tk()
            root.withdraw()
            input_file = filedialog.askopenfilename(title="엑셀 파일 선택", filetypes=[("Excel files", "*.xlsx;*.xls")])
            if not input_file:
                print("파일이 선택되지 않았습니다.")
                return
            
            print(f"선택된 파일: {input_file}")
            wb = openpyxl.load_workbook(input_file)
            ws = wb.active
            
            keyword_column_letter = simpledialog.askstring("입력", "검색어가 포함된 열 (예: B):").upper()
            result_column_letter = simpledialog.askstring("입력", "결과를 저장할 열 (예: D):").upper()
            
            keyword_column_index = openpyxl.utils.column_index_from_string(keyword_column_letter)
            result_column_index = openpyxl.utils.column_index_from_string(result_column_letter)
            
            ws.cell(row=1, column=result_column_index, value="월간검색량")
            
            keywords = [ws.cell(row=i, column=keyword_column_index).value for i in range(2, ws.max_row + 1) if ws.cell(row=i, column=keyword_column_index).value]
            print(f"총 {len(keywords)}개의 키워드를 검색합니다.")
            
            search_volumes = self.get_naver_search_volume(keywords)
            
            for i in range(2, ws.max_row + 1):
                keyword = ws.cell(row=i, column=keyword_column_index).value
                if keyword:
                    ws.cell(row=i, column=result_column_index, value=search_volumes.get(keyword, SearchVolume(0)).total)
            
            wb.save(input_file)
            print(f"검색 결과가 저장되었습니다: {input_file}")
            os.startfile(input_file)  # 파일 자동 열기
        except Exception as e:
            logging.error(f"엑셀 처리 중 오류 발생: {str(e)}")
            print(f"오류 발생: {str(e)}")

if __name__ == "__main__":
    config = NaverAPIConfig(
        API_KEY="0100000000f7219646b1189747046631aa4d35e62954247e750a216e37f7072b5174cbe5c2",
        SECRET_KEY="AQAAAAD3IZZGsRiXRwRmMapNNeYpncLjvqyOrRpn2cZDTV49Ug==",
        CUSTOMER_ID="373124"
    )
    
    search_app = NaverSearchVolume(config)
    search_app.process_excel()

파일이 선택되지 않았습니다.
